# **0. Imports**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint

import pandas as pandas
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

# **1. Hyperparameters**

In [3]:
img_height = 200
img_width = 267
img_channels = 3

learning_rate = 1e-7
batch_size = 5

route = "/content/drive/MyDrive/TFM/data/merged_data" 
#route = "/content/drive/MyDrive/TFM/data_reduced" 

database_len_train = 25195 #580       #total: 25196
database_len_val = 3140 #100          #total: 3148
database_len_test = 3140              #total: 3148

nb_epoch = 1

# Number of batches
steps_per_epoch = np.ceil( database_len_train / batch_size )
validation_steps = np.ceil( database_len_val / batch_size )
test_steps = np.ceil( database_len_test / batch_size )

# **2. Dataset preparation**

In [4]:
# This function returns the generator of the form: (inputs, targets)               
def load_data(Train_df, idx, batch_size):
        
    parking_import = pandas.read_csv( Train_df + "/groundtruth.txt", skiprows=idx*batch_size, nrows=batch_size, delim_whitespace=True)
    parking_data = np.array(parking_import.values)

    batch_images = []
    batch_labels = []  

    for i in range (len(parking_data)):    
        image_name = parking_data[i][0]            
        image_name =  Train_df + "/images/" + image_name        
        img = load_img(image_name, color_mode="rgb", target_size=(img_height, img_width), interpolation="nearest")     
        img = img_to_array(img)        
        batch_images.append(img)         
        batch_labels.append(parking_data[i][1:])
    
    batch_images = np.array(batch_images)
    batch_images = batch_images / 255.0
    batch_labels = np.array(batch_labels).astype('float32')
    
    return (batch_images, batch_labels)           


def batch_generator(directory, batch_size, steps):    
    idx = 1
    while True:
        
        yield load_data(directory, idx-1, batch_size)## Yields data
        
        if idx < steps:
            idx+=1
        else:
            idx=1
            
### Generator objects for train and validation
my_training_batch_generator = batch_generator( route + "/train", batch_size, steps_per_epoch)
my_validation_batch_generator = batch_generator(route + "/val", batch_size, validation_steps)


   

# **3. Model definition**

In [5]:
##################### RESNET 50 MODEL #########################################
img_input = Input(shape=(img_height, img_width, img_channels)) 


model = ResNet50( include_top=False, weights='imagenet',  input_tensor = img_input) 
#model = keras.applications.Xception( include_top=False, weights='imagenet', input_tensor=img_input)
#model = keras.applications.NASNetLarge( include_top=False, weights='imagenet', input_tensor=img_input)

x = model.output

# FC layers
x = Flatten()(x)
x = Dense(1024)(x)
x = Activation('sigmoid')(x)
x = Dropout(0.5)(x)

# Output dimension (empty place probability)
output_dim = 1

x1 = Dense(output_dim)(x)
x1 = Activation('sigmoid', name='a1')(x1)

x2 = Dense(output_dim)(x)
x2 = Activation('sigmoid', name='a2')(x2)
  
x3 = Dense(output_dim)(x)
x3 = Activation('sigmoid', name='a3')(x3)
  
x4 = Dense(output_dim)(x)
x4 = Activation('sigmoid', name='a4')(x4)
  
x5 = Dense(output_dim)(x)
x5 = Activation('sigmoid', name='a5')(x5)
  
x6 = Dense(output_dim)(x)
x6= Activation('sigmoid', name='a6')(x6)
  
x7 = Dense(output_dim)(x)
x7 = Activation('sigmoid', name='a7')(x7)
  
x8 = Dense(output_dim)(x)
x8 = Activation('sigmoid', name='a8')(x8)
  
x9 = Dense(output_dim)(x)
x9 = Activation('sigmoid', name='a9')(x9)
  
x10 = Dense(output_dim)(x)
x10 = Activation('sigmoid', name='a10')(x10)
  
x11 = Dense(output_dim)(x)
x11 = Activation('sigmoid', name='a11')(x11)
  
x12 = Dense(output_dim)(x)
x12 = Activation('sigmoid', name='a12')(x12)
  
x13= Dense(output_dim)(x)
x13 = Activation('sigmoid', name='a13')(x13)
  
x14 = Dense(output_dim)(x)
x14 = Activation('sigmoid', name='a14')(x14)
  
x15 = Dense(output_dim)(x)
x15 = Activation('sigmoid', name='a15')(x15)
  
x16 = Dense(output_dim)(x)
x16 = Activation('sigmoid', name='a16')(x16)
  
x17 = Dense(output_dim)(x)
x17 = Activation('sigmoid', name='a17')(x17)
  
x18 = Dense(output_dim)(x)
x18 = Activation('sigmoid', name='a18')(x18)
  
x19 = Dense(output_dim)(x)
x19 = Activation('sigmoid', name='a19')(x19)
  
x20 = Dense(output_dim)(x)
x20 = Activation('sigmoid', name='a20')(x20)
  
x21 = Dense(output_dim)(x)
x21 = Activation('sigmoid', name='a21')(x21)

model = Model(inputs=[img_input], outputs= [x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21])

#print(model.summary())
#tf.keras.utils.plot_model( model, to_file="model.png", show_shapes=False, show_dtype=False, show_layer_names=True, rankdir="TB", expand_nested=False, dpi=96, layer_range=None,)



94781440/94765736 [==============================] - 1s 0us/step


# **4. Train model**

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=1e-6)

model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['binary_accuracy'], 
              loss_weights=np.ones((21,)).tolist() )

checkpoint_filepath = '/content/drive/MyDrive/TFM/checkpoints/'
#metric = 'val_accuracy' #mode='max' 
metric = 'val_loss' #mode='min'
model_checkpoint_callback = ModelCheckpoint( filepath=checkpoint_filepath,
                                              save_weights_only=True,
                                              monitor=metric,
                                              mode='min',
                                              save_best_only=True)
                          
history = model.fit( my_training_batch_generator,
                    epochs = nb_epoch,
                    steps_per_epoch = steps_per_epoch,
                    verbose = True, 
                    validation_data = my_validation_batch_generator,
                    validation_steps = validation_steps,
                    callbacks=[model_checkpoint_callback])

model.load_weights(checkpoint_filepath)

In [ ]:
#Plot Loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')


In [ ]:
#Plot results
#plt.plot(history.history['binary_accuracy'], label='binary_accuracy')
#plt.plot(history.history['val_binary_accuracy'], label = 'val_binary_accuracy')
#plt.xlabel('Epoch')
#plt.ylabel('Accuracy')

#plt.legend(loc='lower right')

# **5. Test model**

In [ ]:
# Test evaluation
my_test_batch_generator = batch_generator( route + "/test", batch_size, steps_per_epoch)

a = model.evaluate_generator( my_test_batch_generator, 
                    test_steps, 
                    max_queue_size = 10,
                    workers = 1)

print(a)



#read gt_test
#parking_test_import = pandas.read_csv( route + "/test/groundtruth.txt", delim_whitespace=True)
#parking_test_data = np.array(parking_test_import.values)

#parking_test_images = []
#parking_test_labels = []
#for i in range (len(parking_test_data)):
#    image_name = parking_test_data[i][0]            
#    image_name =  route + '/test/images/' + image_name        
#    img = load_img(image_name, color_mode="rgb", target_size=(img_height, img_width), interpolation="nearest")     
#    img = img_to_array(img)        
#    parking_test_images.append(img) 

#    parking_test_label =  parking_test_data[i][1:]
#    parking_test_label = [np.array([i]).astype('float32').reshape((-1,1)) for i in parking_test_label]
#    parking_test_labels.append(parking_test_label)

#test_images = np.array(parking_test_images)
#test_images = test_images / 255.0

#parking_test_labels = np.array(parking_test_labels)


#test_loss, test_acc = model.evaluate(test_images,  parking_test_labels, verbose=2)
#print('Test accuracy: ',test_acc)
#parking_test_labels = np.asarray(parking_test_labels).astype('float32').reshape((-1,1))

# **6. Predict example**

In [ ]:
img = load_img('/content/drive/MyDrive/TFM/data/merged_data/test/images/110GOPRO-GOPR4496.JPG'
               , color_mode="rgb", 
               target_size=(img_height, img_width), 
               interpolation="nearest")     

img = img_to_array(img) 
#Expand dimentions (batch = 1) for a single image since model expects a batch
img = np.expand_dims(img, axis=0)
print(img.shape) 

predictions = model.predict(img, batch_size=None)
print(predictions) 
print(predictions[0]) 
print(predictions[0][0]) 

parking_test_labels = np.asarray(parking_test_labels).astype('float32').reshape((-1,1))

print(parking_test_labels)

